# View sample predictions

Trained model from checkpoint

In [1]:
import argparse
import random
import sys
import warnings

from transformers import MT5Tokenizer
import pandas as pd

sys.path.append('..')  # Allow import of project packages
from text_summarizer.data.motions_data_module import SweParliamentMotionsDataModule
from text_summarizer.models import t5
from text_summarizer.lit_models import t5LitModel
from text_summarizer.util import summarize

/home/erik/miniconda3/envs/swe-parl-mot-summarization/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/erik/miniconda3/envs/swe-parl-mot-summarization/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [2]:
dataset = SweParliamentMotionsDataModule(args=argparse.Namespace())
dataset.prepare_data()
dataset.setup()
print(dataset)

Filtered 618 rows with missing values.
Filtered 7442 texts shorter than 150 characters.
Number of rows remaining: 159722


In [3]:
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', 500)
 
MODEL_PATH = "../training/logs/lightning_logs/version_16/checkpoints/epoch=002-val_loss=0.000-val_cer=0.000.ckpt"
RANDOM_STATE = 1

### Load model

In [4]:
model = t5(
    data_config=dataset.config(),
    args=argparse.Namespace()
)

lit_model = t5LitModel.load_from_checkpoint(
    checkpoint_path=MODEL_PATH,
    model=model,
    args=argparse.Namespace())
lit_model.freeze()

#tokenizer = dataset.data_test.tokenizer

RuntimeError: Error(s) in loading state_dict for t5LitModel:
	Missing key(s) in state_dict: "model.model.encoder.block.0.layer.1.DenseReluDense.wi_0.weight", "model.model.encoder.block.0.layer.1.DenseReluDense.wi_1.weight", "model.model.encoder.block.1.layer.1.DenseReluDense.wi_0.weight", "model.model.encoder.block.1.layer.1.DenseReluDense.wi_1.weight", "model.model.encoder.block.2.layer.1.DenseReluDense.wi_0.weight", "model.model.encoder.block.2.layer.1.DenseReluDense.wi_1.weight", "model.model.encoder.block.3.layer.1.DenseReluDense.wi_0.weight", "model.model.encoder.block.3.layer.1.DenseReluDense.wi_1.weight", "model.model.encoder.block.4.layer.1.DenseReluDense.wi_0.weight", "model.model.encoder.block.4.layer.1.DenseReluDense.wi_1.weight", "model.model.encoder.block.5.layer.1.DenseReluDense.wi_0.weight", "model.model.encoder.block.5.layer.1.DenseReluDense.wi_1.weight", "model.model.encoder.block.6.layer.0.SelfAttention.q.weight", "model.model.encoder.block.6.layer.0.SelfAttention.k.w

### Load data

In [4]:
def summarize(model, text, tokenizer, text_max_num_tokens, summary_max_num_tokens):
    text_encoding = tokenizer(
        text,
        max_length=text_max_num_tokens,
        padding="max_length",
        truncation=True,
        return_attention_mask=True,
        add_special_tokens=True,
        return_tensors="pt",
    )

    generated_ids = model.model.generate(
        input_ids=text_encoding["input_ids"],
        attention_mask=text_encoding["attention_mask"],
        max_length=summary_max_num_tokens,
        num_beams=2,
        repetition_penalty=5.0,
        length_penalty=2.0,
        early_stopping=True,
    )

    preds = [
        tokenizer.decode(
            gen_id, skip_special_tokens=True, clean_up_tokenization_spaces=True
        )
        for gen_id in generated_ids
    ]

    return "".join(preds)

In [5]:
def show_sample_pred():
    sample_index = random.randint(0, 100_000)
    text = dataset.data_test.data[sample_index]
    true_summary = dataset.data_test.targets[sample_index]
    model_summary = summarize(
        model=model,
        text=text,
        tokenizer=tokenizer,
        text_max_num_tokens=512,
        summary_max_num_tokens=64
    )
    print("Motion text:")
    print(50*"-")
    print(text[:500])
    print(50*"-")
    print("Actual title:")
    print(true_summary)
    print(50*"-")
    print("Predicted title:")
    print(model_summary)
    print(50*"-")


In [6]:
show_sample_pred()


NameError: name 'model' is not defined

In [7]:
model = t5(
    data_config=dataset.config(),
    args=argparse.Namespace()
)

lit_model = t5LitModel(
    model=model,
    args=argparse.Namespace())
lit_model.freeze()

tokenizer = dataset.data_test.tokenizer

In [10]:
show_sample_pred()

Motion text:
--------------------------------------------------
Stort behov av fler medicinskt sakkunniga i fibromyalgiFibromyalgi drabbar alltfler och allt längre ner i åldrarna. Debut i 30- till 40- årsåldern blir allt vanligare. Även ännu yngre kan insjukna. Det är fortfarande mest kvinnor som drabbas av fibromyalgi. Man beräknar att närmare 3,5 procent av alla kvinnor har sjukdomen. Dessa unga eller medelålders kvinnor är mycket angelägna att kunna fortsätta att arbeta.Det är därför viktigt att de får en snabb diagnos, lämplig behandling och goda möjligh
--------------------------------------------------
Actual title:
Sakkunniga i fibromyalgi
--------------------------------------------------
Predicted title:
<extra_id_0> i vårt land. Stockholm den
--------------------------------------------------
